<a href="https://colab.research.google.com/github/HollyIrvine/mgmt590_Team_DataNinja7_FinalProject/blob/main/Individual_Analyses/Financial_Analyst/DarrellSanson_DIVE_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dive Analysis**

D - Discover

What is the impact of discounts on profitability?

In [ ]:
#FA_Discover_cost_patterns_based_on_discounts
%%bigquery --project mgmt599-final-project
SELECT
    Discount,
    COUNT(OrderID) AS TotalOrders,
    SUM(Sales) AS TotalRevenue,
    SUM(Profit) AS TotalProfit
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
GROUP BY
    Discount
ORDER BY
    Discount ASC;

In [ ]:
#FA_Discover_profitability_by_category
%%bigquery --project mgmt599-final-project

SELECT
    Category,
    SUM(Profit) AS TotalProfit,
    SUM(Sales) AS TotalSales,
    AVG(Profit) AS AvgProfitPerSale
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
GROUP BY
    Category
ORDER BY
    TotalProfit DESC;

In [ ]:
# FA_Discover_revenue_trends_over_time

%%bigquery --project mgmt599-final-project

SELECT
    DATE_TRUNC(DATE(OrderDate), MONTH) AS OrderMonth,
    SUM(Sales) AS TotalRevenue
FROM
    `mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable`
GROUP BY
    OrderMonth
ORDER BY
    OrderMonth ASC;

Discover Summary

Discounts over 20% lead to negative average profitability, with deeper discounts (e.g., 30%-80%) yielding increasingly severe losses.
Discounting erodes profitability at high levels, suggesting a need for strategic adjustments to discount policies to avoid losses.
Discounts greater than 20% lead to negative profitability across various categories, totaling a loss of 135,369. Discounts over 20% represent 15.79% of the total sales but consistently result in negative profits. Conversely, discounts less than 20% contribute to 84.21% of total sales, yielding positive profitability with a total profit of 422,306. This highlights a correlation between higher discounts and financial losses, suggesting that high discounting adversely impacts overall profitability.

Suggestions for Elimination or Adjustment:
Revisit the >20% Discount Policy:
Identify products where discounts significantly surpass profit margins. Consider reducing discounts on Furniture (Tables, Chairs, Bookcases, Furnishings), technology products (Machines, Phones), and office supplies (Binders, Appliances).
Stop selling items with persistent losses:
Evaluate whether to discontinue categories that consistently result in losses, such as Office Supplies - Appliances and Furniture - Furnishings.
Optimize Pricing Strategy:
Reassess price points and market competitiveness for items underperforming with high discount levels.
Focus on Products with Positive Profitability:
Categories under <20% Discounts (e.g., Copiers, Phones, Accessories, Paper, Storage, Binders) show strong profitability; prioritize sales and marketing efforts towards these products.



In [ ]:
#FA_Extend_future_profitability_projection_by_category
%%bigquery --project mgmt599-final-project

SELECT
    Category,
    SUM(Sales) AS HistoricalSales,
    SUM(Profit) AS HistoricalProfit,
    AVG(Profit) * 1.10 AS ProjectedProfit
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
GROUP BY
    Category
ORDER BY
    HistoricalProfit DESC;



### I - Investigate

How does profitability vary across customer segments for discounts above 20%

In [ ]:
# FA_Investigate_Impact_of_customer_segments_on_sales_and_profit

%%bigquery --project mgmt599-final-project
SELECT
    Segment,
    SUM(Sales) AS TotalSales,
    SUM(Profit) AS TotalProfit,
    COUNT(OrderID) AS TotalOrders
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
GROUP BY
    Segment
ORDER BY
    TotalProfit DESC;

In [ ]:
#FA_Investigate_Impact_of_discounts_on_profitability
%%bigquery --project mgmt599-final-project
SELECT
    Discount,
    SUM(Sales) AS TotalSales,
    SUM(Profit) AS TotalProfit,
    AVG(Profit) AS AvgProfit
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
GROUP BY
    Discount
ORDER BY
    Discount ASC;

In [ ]:
#FA_Investigate_regional_performance_insights
%%bigquery --project mgmt599-final-project

SELECT
    Region,
    SUM(Sales) AS TotalSales,
    SUM(Profit) AS TotalProfit,
    COUNT(OrderID) AS TotalOrders
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
GROUP BY
    Region
ORDER BY
    TotalProfit DESC;

I - Investigate Summary

Patterns reveal that:
Discounts across segments and categories hit profitability universally, but Furniture and Consumers seem most affected.
Fixed costs seem unmanageable at higher discount levels, implying inefficiencies in cost structure for discounted sales.


### V - Validate

What could make this conclusion wrong?
If customers incentivized by high discounts purchase more in the long term, the negative profitability on current transactions could eventually balance out.
What data limitations exist?
Lack of data on customer lifetime value (CLV) post-discount prevents understanding of long-term outcomes.
Limited visibility into variable costs for different categories could mean cost structures are uneven.


In [ ]:
#FA_Validate_testing_correlation_between_sales_and_profit
%%bigquery --project mgmt599-final-project
SELECT
    Sales,
    Profit
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
WHERE
    Sales > 0 AND Profit > 0
ORDER BY
    Sales DESC;

In [ ]:

#FA_Validate_Segment_and_discount_correlation_with_profitability
%%bigquery --project mgmt599-final-project

SELECT
    Segment,
    Discount,
    AVG(Profit) AS AvgProfitPerOrder,
    SUM(Profit) AS TotalProfit,
    COUNT(OrderID) AS TotalOrders
FROM
    mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
GROUP BY
    Segment, Discount
ORDER BY
    AvgProfitPerOrder DESC;

# E - Extend

Recommendations:
Adjust Discount Policy:
Cap discounts at 20%: Avoid deeper discounts that consistently result in unprofitable transactions. Maintain profitability margins.
Category-Specific Adjustments:
Technology: Continue targeted discounts for high-margin products with measured increases in discount levels.
Furniture: Reduce reliance on discounts as marketing drivers; focus on retention strategies based on customer loyalty.
Foster Customer Loyalty Without Deep Discounts:
Implement loyalty programs to encourage repeat purchases with smaller incentives.
Offer personalized discounts to Corporate customers, leveraging higher average profit per customer.


In [ ]:
# FA_Extend_Projected_sales_based_on_historical_growth

%%bigquery --project mgmt599-final-project

WITH MonthlyRevenue AS (
    SELECT
        DATE_TRUNC(OrderDate, MONTH) AS OrderMonth,  -- Ensure "OrderDate" is of type DATE/TIMESTAMP
        SUM(Sales) AS TotalRevenue
    FROM
        mgmt599-final-project.EcomStoreSalesPipeline.EcomStoreDataTable
    GROUP BY
        DATE_TRUNC(OrderDate, MONTH)  -- Correct DATE_TRUNC syntax for BigQuery
)
SELECT
    OrderMonth,
    TotalRevenue,
    LAG(TotalRevenue) OVER (ORDER BY OrderMonth) AS PreviousMonthRevenue,
    (TotalRevenue - LAG(TotalRevenue) OVER (ORDER BY OrderMonth)) AS RevenueChange,
    ROUND(((TotalRevenue - LAG(TotalRevenue) OVER (ORDER BY OrderMonth)) / LAG(TotalRevenue) OVER (ORDER BY OrderMonth)) * 100, 2) AS RevenueGrowthPercent
FROM
    MonthlyRevenue;